In [1]:
!which python

/Users/tareksanger/Library/Caches/pypoetry/virtualenvs/lumis-tzzr_5k5-py3.12/bin/python


In [2]:
from lumis.config import config
import os 
os.environ['OPENAI_API_KEY'] = config.openai_api_key

In [6]:
from lumis.llm.ChatCompletion import ChatCompletion
from pydantic import BaseModel, Field
from enum import Enum

class Action(str, Enum):
  "The type of action to be performed."
  search = "search"
  lookup = "lookup"
  finish = "finish"


class Thought(BaseModel):
    "The action to be performed."
    thought: str = Field(..., description="You're thought regarding taking the action.", max_length=100)
    action: Action = Field(..., description="The type of action to be performed.")

class SearchAction(BaseModel):
    "The Search query to be preformed."
    entity: str = Field(
        ..., description="The entity to search on Wikipedia.", max_length=100
    )

class LookupAction(BaseModel):
    "The Lookup query to be preformed."
    keyword: str = Field(
        ..., description="The keyword to lookup in the current passage.", max_length=100
    )

class FinishAction(BaseModel):
    "The Finish query to be preformed."
    answer: str = Field(..., description="The answer to the question.", max_length=100)


system_content = """
Solve a question answering task with interleaving Thought, Action, Observation steps. Thought can reason about the current situation, and Action can be three types: 
(1) Search[entity], which searches the exact entity on Wikipedia and returns the first paragraph if it exists. If not, it will return some similar entities to search.
(2) Lookup[keyword], which returns the next sentence containing keyword in the current passage.
(3) Finish[answer], which returns the answer and finishes the task.

Choose one of the provided functions to perform the action.
"""

prompt = ChatCompletion(user_content="Can you tell me about Tony Soprano?", system_content=system_content, functions=[Thought, SearchAction, LookupAction, FinishAction])
completion = prompt.create()

In [7]:
import json
import pprint

response = json.loads(completion.choices[0].message.function_call.arguments)

pprint.pprint(response, width=250)

{'action': 'search', 'thought': 'I should search for information about Tony Soprano.'}


In [ ]:
prompt = ChatCompletion(user_content="", system_content=system_content, functions=[Thought, SearchAction, LookupAction, FinishAction])

In [ ]:
import requests
import pprint
import time
from bs4 import BeautifulSoup

class WikiSearch:

    def __init__(self, search_query):
        self.search_query = search_query

    def search(self):
        url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={self.search_query}&format=json"
        response = requests.get(url)
        return response.json()

    def search2(self):
        search_url = f"https://en.wikipedia.org/w/index.php?search={self.search_query}"
        # old_time = time.time()
        response_text = requests.get(search_url).text
        return response_text

In [ ]:
import re
wiki = WikiSearch("Stantec")


In [ ]:
result = wiki.search2()


soup = BeautifulSoup(result, features="html.parser")

text = ""
for paragraph in soup.find_all("p"):
    text += paragraph.text

# Preprocessing the data
text = re.sub(r"\[[0-9]*\]", " ", text)
# text = re.sub(r"\s+", " ", text)
# text = text.lower()
# text = re.sub(r"\d", " ", text)

pprint.pprint(text)

('Stantec Inc. is an international professional services company in the design '
 'and consulting industry. The company was founded in 1954, as D. R. Stanley '
 'Associates in Edmonton, Alberta, Canada. Stantec provides professional '
 'consulting services in planning, engineering, architecture, interior design, '
 'landscape architecture, surveying, environmental sciences, project '
 'management, and project economics for infrastructure and facilities '
 'projects. The company provides services on projects around the world, with '
 'over 30,000 employees operating out of more than 450 locations in North '
 'America and across offices on six continents. \n'
 'Don Stanley was the first Canadian to earn a Ph.D. in environmental '
 'engineering.  Attending Harvard University on a Rockefeller Foundation '
 'scholarship, he earned his doctorate in 1952 and two years later founded '
 'D.R. Stanley & Associates, working as the sole proprietor out of a '
 '250-square-foot (23\xa0m2) office in 